Import Questions 1 Dataframe

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

raw_data = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641')[0]
filtered = raw_data[raw_data['Borough'] != 'Not assigned']
filtered['Neighbourhood'].replace('Not assigned', filtered['Borough'], inplace=True)
# Reset index
filtered.reset_index(drop=True, inplace=True)
for i in range(len(filtered)): 
    # Break loop if i is as long as the length. 
    # Because we continiously drop rows and reset indexs this is required otherwise it will try a value for i that is longer than the length of the dataframe
    if (i==len(filtered)):
        break
    
    # For all rows which have duplicate Postcodes
    if (filtered.loc[filtered['Postcode'] == filtered['Postcode'][i]].shape[0]) > 1:
        concat_neighbourhood = filtered['Neighbourhood'][i]
        
        # To concatinate the Neighbourhoods strings with all the duplicate postcode rows 
        for x in range(len(filtered.loc[filtered['Postcode'] == filtered['Postcode'][i]])):
            if (x>0):
                dup_group = filtered.loc[filtered['Postcode'] == filtered['Postcode'][i]]
                dup_group.reset_index(drop=True, inplace=True)
                concat_neighbourhood = concat_neighbourhood + ', ' + dup_group['Neighbourhood'][x]
                
        # Set all Neighborhoods for a particular postcode to the concatinated neighborhood string
        filtered.loc[filtered['Postcode'] == filtered['Postcode'][i],'Neighbourhood'] = concat_neighbourhood
        
        # Remove the duplicate rows now that all rows for the postcode we just searched should be the same, reset index
        filtered.drop_duplicates(inplace=True) 
        filtered.reset_index(drop=True, inplace=True)

filtered

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."


Merge with Latitude/Longitude

In [3]:
data = pd.read_csv('http://cocl.us/Geospatial_data')
data.rename(columns={"Postal Code": "Postcode"}, inplace=True)
filtered = pd.merge(filtered, data, on='Postcode')
filtered

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",43.636258,-79.498509
